### Renaming dataset, creating DataFrame with data

In [143]:
from pathlib import Path
import numpy as np
import pandas as pd
import shutil
import os

In [144]:
conf_path = '/home/ezhur/datasets/projects/hakaton_pipes/DATASET/metadata/set.cfg'

with open(conf_path, 'r', encoding='utf-8') as f:
    data = f.read()
    data = data.split('\n')[:-1]

In [145]:
# idxs of lines with pathes
idxs = []
for i, elem in enumerate(data):
    if elem.endswith('.frame'):
        idxs.append(i)

In [146]:
# dict path: list of boxes
last = None
data_dict = dict()
for elem in data:
    if elem.endswith('.frame'):
        data_dict[elem] = []
        last = elem
    else:
        data_dict[last].append(elem)

In [147]:
# df of images with defects
data_list = []

path_num = 0
for k in data_dict.keys():
    path_num += 1
    info = data_dict[k]
    for elem in info:
        x, y, cl = elem.split()
        x = int(x[:-1])
        y = int(y[:-1])
        cl = int(cl)
        data_list.append([k, x, y, cl, path_num])

df = pd.DataFrame(data_list, columns=['path', 'x', 'y', 'class', 'new_path'])

df['path'] = df['path'].apply(lambda x: x[:-6]+'.bmp') 
df['path'] = df['path'].apply(lambda x: x.replace('\\', '/')) 
df['new_path'] = df['new_path'].apply(lambda x: str(x)+'.jpg')

In [148]:
# df with config info
df

,path,x,y,class,new_path
0,0/1538/frame0006.bmp,809,336,3,1.jpg
1,0/1538/frame0006.bmp,601,299,0,1.jpg
2,0/1538/frame0007.bmp,667,318,3,2.jpg
3,0/1538/frame0007.bmp,811,523,3,2.jpg
4,0/1538/frame0007.bmp,445,300,0,2.jpg
...,...,...,...,...,...
38008,2023.10.25/5_896.bmp,121,418,1,7740.jpg
38009,2023.10.25/5_896.bmp,568,94,0,7740.jpg
38010,2023.10.25/5_896.bmp,509,153,0,7740.jpg
38011,2023.10.25/5_896.bmp,509,330,0,7740.jpg


In [149]:
# remove pathes which does not exist
source_root = Path('/home/ezhur/datasets/projects/hakaton_pipes/DATASET/FRAMES')

idx_to_remove = []
for index, row in df.iterrows():
    path = row['path']
    if not (source_root / path).exists():
        idx_to_remove.append(index)

df = df.drop(index=idx_to_remove)
df

,path,x,y,class,new_path
0,0/1538/frame0006.bmp,809,336,3,1.jpg
1,0/1538/frame0006.bmp,601,299,0,1.jpg
2,0/1538/frame0007.bmp,667,318,3,2.jpg
3,0/1538/frame0007.bmp,811,523,3,2.jpg
4,0/1538/frame0007.bmp,445,300,0,2.jpg
...,...,...,...,...,...
38008,2023.10.25/5_896.bmp,121,418,1,7740.jpg
38009,2023.10.25/5_896.bmp,568,94,0,7740.jpg
38010,2023.10.25/5_896.bmp,509,153,0,7740.jpg
38011,2023.10.25/5_896.bmp,509,330,0,7740.jpg


In [150]:
path_num

7740

In [151]:
# obtain list of all images
source_root = Path('/home/ezhur/datasets/projects/hakaton_pipes/DATASET/FRAMES')

img_paths_list = []

for root, dirs, files in os.walk(source_root, topdown=True):
   for name in files:
      path = os.path.join(root, name)
      if path.endswith('.bmp'):
         path = Path(path).relative_to(source_root).as_posix()
         img_paths_list.append(path)

In [152]:
# imgs which are not in config file
img_paths_list_without_defect = \
    list(set(img_paths_list).difference(set(df['path'].values)))

In [153]:
# df of imgs with no defect
df_no_bbox = pd.DataFrame(img_paths_list_without_defect, columns=['path'])
df_no_bbox['x'] = np.nan
df_no_bbox['y'] = np.nan
df_no_bbox['class'] = np.nan
df_no_bbox['new_path'] = df_no_bbox.index
df_no_bbox['new_path'] = df_no_bbox['new_path'].apply(lambda x: str(x+path_num+1)+'_no_def.jpg')
df_no_bbox

,path,x,y,class,new_path
0,2020.10.26/1_583.bmp,NaN,NaN,NaN,7741_no_def.jpg
1,2023.04.18/0/4_117.bmp,NaN,NaN,NaN,7742_no_def.jpg
2,2021.05.27/0_869.bmp,NaN,NaN,NaN,7743_no_def.jpg
3,2023.04.18/0/3_116.bmp,NaN,NaN,NaN,7744_no_def.jpg
4,2020.10.17/Труба 1/0_830.bmp,NaN,NaN,NaN,7745_no_def.jpg
...,...,...,...,...,...
962,2020.10.25/3_495.bmp,NaN,NaN,NaN,8703_no_def.jpg
963,0/414/frame0003.bmp,NaN,NaN,NaN,8704_no_def.jpg
964,0/2070/frame0000.bmp,NaN,NaN,NaN,8705_no_def.jpg
965,0/mv_1/frame0011.bmp,NaN,NaN,NaN,8706_no_def.jpg


In [154]:
df = pd.concat([df, df_no_bbox], ignore_index=True)

In [155]:
df

,path,x,y,class,new_path
0,0/1538/frame0006.bmp,809.0,336.0,3.0,1.jpg
1,0/1538/frame0006.bmp,601.0,299.0,0.0,1.jpg
2,0/1538/frame0007.bmp,667.0,318.0,3.0,2.jpg
3,0/1538/frame0007.bmp,811.0,523.0,3.0,2.jpg
4,0/1538/frame0007.bmp,445.0,300.0,0.0,2.jpg
...,...,...,...,...,...
34539,2020.10.25/3_495.bmp,NaN,NaN,NaN,8703_no_def.jpg
34540,0/414/frame0003.bmp,NaN,NaN,NaN,8704_no_def.jpg
34541,0/2070/frame0000.bmp,NaN,NaN,NaN,8705_no_def.jpg
34542,0/mv_1/frame0011.bmp,NaN,NaN,NaN,8706_no_def.jpg


In [156]:
df.to_csv('./df_pipe_paths.csv', index=0)

In [157]:
# only unique pathes
df_pathes = df[['path', 'new_path']].drop_duplicates()
df_pathes

,path,new_path
0,0/1538/frame0006.bmp,1.jpg
2,0/1538/frame0007.bmp,2.jpg
7,0/1538/frame0009.bmp,3.jpg
10,0/1538/frame0011.bmp,4.jpg
13,0/1538/frame0012.bmp,5.jpg
...,...,...
34539,2020.10.25/3_495.bmp,8703_no_def.jpg
34540,0/414/frame0003.bmp,8704_no_def.jpg
34541,0/2070/frame0000.bmp,8705_no_def.jpg
34542,0/mv_1/frame0011.bmp,8706_no_def.jpg


In [158]:
# save imgs with new names
dest_root = Path('/home/ezhur/datasets/projects/hakaton_pipes/dataset_namings/imgs')
source_root = Path('/home/ezhur/datasets/projects/hakaton_pipes/DATASET/FRAMES')

for index, row in df_pathes.iterrows():
    path = row['path']
    new_path = row['new_path']
    if (source_root / path).exists():
        shutil.copy(source_root / path, dest_root / new_path)